In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.impute import KNNImputer

In [ ]:
vc1 = pd.read_csv('../input/solar-generation-miniproject/canopyfinal.csv', error_bad_lines=False)
vc2 = pd.read_csv('../input/solar-generation-miniproject/groundfinal.csv', error_bad_lines=False)
vc3 = pd.read_csv('../input/solar-generation-miniproject/rooffinal.csv', error_bad_lines=False)
vc3['RefCell1_Wm2_Avg']=vc3['RefCell1_Wm2_Avg']/3
vc1['RefCell1_Wm2_Avg']=vc1['RefCell1_Wm2_Avg']/2


In [ ]:
vtnt=pd.concat([vc1,vc2,vc3],axis=0)

In [ ]:
btnt=vtnt.dropna()

In [ ]:
vtnt

In [ ]:
X = btnt.iloc[:, 0:-2]
y = btnt.iloc[:, -1]
X = X.values
y = y.values



In [ ]:
y

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test=np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))



regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error')
#filepath="weights.best.hdf5"
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

regressor.fit(X_train, y_train,validation_split=0.1, epochs = 15, batch_size = 32,callbacks=callbacks_list)


In [ ]:

pred = regressor.predict(X_test)



In [ ]:
from sklearn.externals import joblib
joblib_file = "./joblib_RL_Model.pkl"  
joblib.dump(regressor, joblib_file)

In [ ]:
pred

In [ ]:
np.sum(y_test, dtype = np.float32)

In [ ]:
np.sum(pred, dtype = np.float32)